# Grating Couplers

Grating couplers are simply components of a photonic circuit that use refraction to couple light into a waveguide. By utilizing refraction, we can avoid have to use expensive specialized equipment to send light directly through the edge of the chip and we get the added versatility of being able input light to any location on the chip.

![Image of an electromagnetic wave](https://raw.githubusercontent.com/BYUCamachoLab/Photonics-Bootcamp/main/book/images/Notebook_Images/grating_coupler_3d.png)
{cite}`Cheng_Mao_Li_Han_Fu_2020`